# Hist Design Prototype

This is `fill` method in python loop:

In [ ]:
import numpy as np
import numba as nb
from hist import Hist
from hist import axis

array = np.random.randn(10000)
h = Hist.new.Reg(100, -3, 3, name="x", label="x-axis").Double()

## Numba: Hist

To extend the Numba, we first need to create a Hist type `HistType` for `Hist`, and then teach Numba about our type inference additions:

In [ ]:
from numba import types
import numba as nb

# create Numba type
class HistType(types.Type):
    arraytype = nb.types.Array(nb.types.float64, 1, "C")

    def __init__(self):
        super().__init__(name="Hist")


hist_type = HistType()

# infer values
@nb.extending.typeof_impl.register(Hist)
def typeof_index(val, c):
    return hist_type


# infer annotations
nb.extending.as_numba_type.register(Hist, hist_type)

# infer operations
@nb.extending.type_callable(Hist)
def type_hist(context):
    def typer(axes):
        for ax in axes:
            # TODO: Assumed all are Regular axes
            if not (isinstance(ax, hist.axis.Regular)):
                return None
        return HistType

    return typer

We also need to teach Numba how to actually generate native representation for the new operations:

In [ ]:
import numba as nb
from numba.core import cgutils
from numba.extending import (
    models,
    overload_attribute,
    lower_builtin,
    NativeValue,
)

# define data model
@nb.extending.register_model(HistType)
class HistModel(models.StructModel):
    def __init__(self, dmm, fe_type):
        members = [
            ("bins", types.int64),
            ("lo", types.float64),
            ("hi", types.float64),
            ("data", fe_type.arraytype),
        ]
        super().__init__(dmm, fe_type, members)


# expose attributes, porperties and constructors
nb.extending.make_attribute_wrapper(HistType, "bins", "bins")
nb.extending.make_attribute_wrapper(HistType, "lo", "lo")
nb.extending.make_attribute_wrapper(HistType, "hi", "hi")
nb.extending.make_attribute_wrapper(HistType, "data", "data")


@nb.extending.lower_builtin(Hist, types.Integer, types.Float, types.Float, types.Array)
def impl_h(context, builder, sig, args):
    typ = sig.return_type
    lo, hi, bins, data = args
    h = cgutils.create_struct_proxy(typ)(context, builder)
    h.lo = lo
    h.hi = hi
    h.bins = bins
    h.data = data
    return h._getvalue()


# unbox and box
@nb.extending.unbox(HistType)
def unbox_h(typ, obj, c):
    # lower = h.axes[0][0][0]
    # upper = h.axes[0][-1][-1]
    # bins = h.axes[0].__len__(self)
    # data = h.values()

    start_obj = c.pyapi.long_from_long(c.context.get_constant(nb.long_, 0))
    stop_obj = c.pyapi.long_from_long(c.context.get_constant(nb.long_, -1))

    data_obj = c.pyapi.call_method(obj, "values")

    axis_tuple_obj = c.pyapi.object_getattr_string(obj, "axes")
    axis_obj = c.pyapi.tuple_getitem(axis_tuple_obj, 0)
    bins_obj = c.pyapi.call_method(axis_obj, "__len__")

    lo1_obj = c.pyapi.object_getitem(axis_obj, start_obj)
    hi1_obj = c.pyapi.object_getitem(axis_obj, stop_obj)

    lo_obj = c.pyapi.tuple_getitem(lo1_obj, 0)
    hi_obj = c.pyapi.object_getitem(hi1_obj, stop_obj)

    h = cgutils.create_struct_proxy(typ)(c.context, c.builder)

    h.bins = c.pyapi.number_as_ssize_t(bins_obj)
    h.lo = c.pyapi.float_as_double(lo_obj)
    h.hi = c.pyapi.float_as_double(hi_obj)
    h.data = c.pyapi.to_native_value(typ.arraytype, data_obj).value

    c.pyapi.decref(bins_obj)
    c.pyapi.decref(lo_obj)
    c.pyapi.decref(hi_obj)
    c.pyapi.decref(data_obj)

    c.pyapi.decref(lo1_obj)
    c.pyapi.decref(hi1_obj)

    c.pyapi.decref(axis_tuple_obj)
    # c.pyapi.decref(axis_obj) - no deref needed, crashes

    c.pyapi.decref(start_obj)
    c.pyapi.decref(stop_obj)

    is_error = cgutils.is_not_null(c.builder, c.pyapi.err_occurred())
    return NativeValue(h._getvalue(), is_error=is_error)

We also need to teach numba about running the fill:

In [ ]:
@nb.extending.overload_method(HistType, "fill")
def fill_resolve(hist, val):
    if not isinstance(hist, HistType):
        return None
    if not isinstance(val, nb.types.Float):
        return None

    def fill(hist, val):
        delta = 1 / ((hist.hi - hist.lo) / hist.bins)
        i = int((val - hist.lo) * delta)

        if 0 <= i < hist.bins:
            hist.data[i] += 1

    return fill

Timing the Python version:

In [ ]:
%%timeit
h_python = h.copy()
h_python.fill(array)

In [ ]:
@nb.njit
def nb_fill_hist(h, v):
    for v in array:
        h.fill(v)

Timing the Numba version:

In [ ]:
%%timeit
h_numba = h.copy()
nb_fill_hist(h_numba, array)

Showing the results:

In [ ]:
h_numba = h.copy()
nb_fill_hist(h_numba, array)
h_numba

In [ ]:
h_python = h.copy()
h.fill(array)